In [1]:
# Installation
%%capture

!pip install transformers==4.5.1
!pip install datasets==1.6.2
!pip install tokenizers==0.10.2
!pip install torch==1.8.1+cu111
!pip install psutil==5.8.0
!pip install rouge_score
!pip install sacrebleu
!pip install openpyxl
!pip install xlrd
!pip install git-python
!pip install -U ipython==7.20
!pip install cmake
!pip install SentencePiece

In [2]:
# Imports
import gc
import csv
import torch
import psutil
import datasets
import transformers
import pandas as pd

from datasets import ClassLabel
from IPython.display import display, HTML

In [3]:
# Drive
from google.colab import drive
drive.mount("/content/drive")
path_drive = "/content/drive/My Drive/Temp/"

Mounted at /content/drive


In [16]:
# Config
language = "german"  # english, german, multilingual
model_name = "bert-base-multilingual-cased"
tokenizer_name = "bert-base-multilingual-cased"
batch_size = 8

ratio_corpus_wiki = 1.00
ratio_corpus_news = 1.00
ratio_corpus_mlsum = 1.00
ratio_corpus_eng = 1.00

path_output = path_drive + "Models/"
path_checkpoint = path_output + "/checkpoint"

corpus_german = [
    """ Die Täter wussten, was sie taten. Die Anschläge waren mediengerecht umgesetzt. Die Symbole, die sie zerstörten, waren präzise ausgewählt: Das weiße Haus und das Pentagon als Symbole der Macht, das World Trade Center als Symbol des Kapitals und der Wirtschaft. 2.750 Menschen, darunter 30 Deutsche, starben in oder an den Türmen des WTC, 147 in den Flugzeugen, 184 Opfer kamen im Pentagon ums Leben, 40 Menschen wurden in dem in Pennsylvania abgestürtzten Flugzeug Opfer des bisher perfidesten Terroranschlags, den islamistische Terroristen ausgelöst haben. Die USA befanden sich im Schockzustand: Die Anschläge waren der erste quasi militärische Angriff auf die Vereinigten Staaten seit 1814. Amerika erwies sich am 11. September 2001 erstmals seit den Kämpfen um seine Unabhängigkeit von England im späten 18. und frühen 19. Jahrhundert als im eigenen Land verwundbar. Die Anschläge hatten daher weltweit gravierende politische und militärische Folgen. Der amerikanische Präsident George W. Bush, der sich zum Zeitpunkt der Anschläge in Florida aufhielt, äußert sich in einer ersten Fernsehansprache: "Täuschen Sie sich nicht. Wir werden diese Leute bis zum Ende jagen und bestrafen." """,
    """ Bundestrainer Joachim Löw sah nach dem famosen 7:1-Triumph im Halbfinale gegen Gastgeber Brasilien keinen Anlass, seine Startelf umzustellen - bis kurz vor Anpfiff. Dann erreichte den Bundestrainer die Hiobsbotschaft: Khedira meldete sich mit Wadenproblemen kurzfristig ab, für ihn rückte Kramer in die Startformation. Das DFB-Team agierte durchaus bemüht, doch zielstrebiger blieben die Südamerikaner. Nach einer Flanke von Lavezzi schob Higuain zum vermeintlichen 1:0 ein, wurde aber wegen einer Abseitsposition korrekterweise zurückgepfiffen. Je länger die Partie dauerte, desto mehr scheuten beide Seiten das Risiko. Mit dem Mut schwand auch das Tempo aus der im ersten Abschnitt noch so rassigen Begegnung. Dafür wurden die Zweikämpfe ruppiger: Mascherano und Aguero kassierten Verwarnungen. Und wenn sich mal unverhofft eine Chance bot, schlich sich ein technischer Fehler ein. Es blieb ein zähes Ringen. Die bereits vorbelasteten Mascherano und Aguero hatten Glück, dass sie nicht vorzeitig mit Gelb-Rot vom Platz mussten, als sie Schweinsteiger erst umsensten und dann eine Platzwunde im Gesicht verschafften. Zwei ließ das kalt: Schürrle zog noch einen beherzten Spurt an und fand Götze im Zentrum, der das Spielgerät mit der Brust runterpflückte und gekonnt mit links volley einschoss - ein Traumtor zu einem ganz wichtigen Zeitpunkt. """,
    """ Die Inflation ist ein anhaltender Prozess der Geldentwertung, der sich durch allgemeine Preiserhöhungen bemerkbar macht. Mit einer Geldeinheit kann dann ständig weniger gekauft werden, d.h. die Kaufkraft des Geldes vermindert sich dauernd. Nicht als Inflation gelten einmalige, vorübergehende, durch ungewöhnliche Vorkommnisse verursachte Preisniveauerhöhungen sowie Preissteigerungen für bestimmte Güter oder Produktionsfaktoren. Die Inflation wird gemessen am Anstieg eines das allgemeine Preisniveau am besten widerspiegelnden Preisindexes wie z.B. des Verbraucherpreisindexes für Deutschland. Der prozentuale Anstieg des Preisindexes in einem bestimmten Zeitraum wird als Inflationsrate bezeichnet. Beim Entstehen einer Inflation spielt besonders die Geldmenge in der Volkswirtschaft eine große Rolle. Steht der gesamtwirtschaftlichen Gütermenge eine zu große Geldmenge gegenüber, ist eine Bedingung für die Inflation gegeben. Übersteigt die gesamtwirtschaftliche Güternachfrage das gesamtwirtschaftliche Güterangebot, das kurzfristig nicht erhöht werden kann, sind steigende Preise die Folge und die Inflation setzt ein. Die Preissteigerungen lösen steigende Löhne aus, wegen des höheren Einkommens steigt die Nachfrage nach Gütern an. Die höheren Löhne bewirken jedoch auch steigende Kosten der Unternehmen, was wiederum zu Preissteigerungen für Güter führt. """,
    """ Storniert ein Reisender wegen einer Virus-Pandemie die Reise, so steht dem Reiseveranstalter kein Anspruch auf Entschädigung gemäß § 651 h Abs. 1 Satz 3 BGB zu, wenn er ebenfalls die Reise absagt. Zudem begründet die fehlende Therapiemöglichkeit und Impfung ein kostenloses Reise­rücktritts­recht. Dies hat das Amtsgericht Stuttgart entschieden. Dem Fall lag folgender Sachverhalt zugrunde: Wenige Tage vor Beginn einer mehrtägigen Flug- und Busrundreise durch Portugal im März 2020 erklärte die Reisende die Stornierung. Sie begründete dies mit dem sich ausbreitenden Corona-Virus. Die Reiseveranstalterin sagte zwei Tage später ebenfalls die Reise ab. Dennoch verlangte sie von der Reisenden die Zahlung einer Stornogebühr. Da sich die Reisende weigerte dem nachzukommen, kam es zu einem Klageverfahren. Das Amtsgericht Stuttgart entschied zu Gunsten der Reisenden. Der Reiseveranstalterin stehe kein Anspruch auf Entschädigung gemäß § 651 h Abs. 1 Satz 3 BGB zu. Denn zum einen könne sich die Reisende auf außergewöhnliche Umstände im Sinne von § 651 h Abs. 3 BGB berufen. Zum anderen sei zu berücksichtigen, dass die Reiseveranstalterin ebenfalls die Reise abgesagt hat. """,
    """ Lege den Kinderwagen zum Aufklappen flach auf den Boden, löse die Gestellsperre an der Seite und ziehe den Griff bis zum Einrasten des Klappmechanismus nach oben. Vergewissere Dich vor dem Gebrauch, ob der Kinderwagen richtig aufgeklappt ist. Um die Hinterachse zu montieren, setze diese samt Bremsanlage auf das Gestell, bis die Hinterachse beidseitig hörbar eingerastet ist. Halte den Druckknopf in der Radnabe gedrückt und ziehe das Rad gleichzeitig von der Achse, um die Hinterräder zu demontieren und ziehe das Rad von der Achse. Zur neuerlichen Montage drücke den Druckknopf und schiebe das Rad bis zum Einrasten auf die Achse. Versichere Dich vor dem Gebrauch, ob alle Räder richtig montiert sind. Der Kinderwagen ist mit einer Standbremse an der Hinterachse ausgestattet. Betätige das Bremspedal, um die Räder zu blockieren. Hebe das Pedal an, um die Bremse zu lösen. Blockiere die Bremse immer, wenn Du den Kinderwagen auf unebenem Untergrund abstellst. Die Bremse dient nicht zur Verlangsamung bei der Fahrt und ist ausschließlich bei völligem Stillstand des Kinderwagens zu blockieren. Der Kinderwagen ist nicht zum Laufen geeignet. """,
    """ Algorithmen begegnen uns täglich sowohl auf der Arbeit als auch in der Freizeit und sind aus unserem modernen Leben nicht mehr wegzudenken. Meist hilfreich aber auch nicht immer unbedenklich, kommen Algorithmen immer größere Bedeutung zu. Was ein Algorithmus ist und wie sie unser Leben prägen, wird in dem folgenden Artikel erläutert. Allgemein gesagt, gibt ein Algorithmus eine Vorgehensweise vor, um ein Problem zu lösen. Anhand dieses Lösungsplans werden in Einzelschritten Eingabedaten in Ausgabedaten umgewandelt. Besonders in der Informatik spielen Algorithmen eine große Rolle. Sie stellen eine Grundlage der Programmierung dar und sind unabhängig von einer konkreten Programmiersprache. Trotzdem sind Algorithmen nicht nur in der Informatik oder Mathematik vorzufinden. Denn Algorithmen werden nicht nur maschinell durch einen Rechner ausgeführt sondern können auch von Menschen in natürlicher Sprache formuliert und abgearbeitet werden. Unser modernes Leben ist abhängig von Algorithmen, ohne dass uns dies immer bewusst ist. Die Einsatzgebiete von Algorithmen sind sehr vielfältig: Im Navi zeigen sie uns den kürzesten Weg, schlagen uns als Computergegner im Schach, kontrollieren unseren Satzbau in Office Word oder empfehlen uns einen passenden Partner beim Online-Dating. Ein Algorithmus steckt in vielen technischen Geräten sowie hinter unserer elektronischen Kommunikation. """
]

corpus_english = [
    """ The perpetrators knew what they were doing. The attacks were media-ready. The symbols they destroyed were precisely chosen: The White House and the Pentagon as symbols of power, the World Trade Center as a symbol of capital and business. 2,750 people, including 30 Germans, died in or on the towers of the WTC, 147 in the airplanes, 184 victims lost their lives in the Pentagon, 40 people were killed in the plane that crashed in Pennsylvania in the most perfidious terrorist attack yet unleashed by Islamist terrorists. The U.S. was in a state of shock: The attacks were the first quasi-military attack on the United States since 1814. America proved vulnerable at home on Sept. 11, 2001, for the first time since the struggles for its independence from England in the late 18th and early 19th centuries. As a result, the attacks had serious political and military consequences around the world. U.S. President George W. Bush, who was in Florida at the time of the attacks, made the following remarks in an initial televised address: "Make no mistake. We will hunt these people down and punish them to the end." """,
    """ German coach Joachim Löw saw no reason to change his starting eleven after the fantastic 7-1 triumph in the semifinals against host Brazil - until shortly before kickoff. Then the coach received bad news: Khedira had to pull out at short notice with calf problems, and Kramer was replaced in the starting lineup. The DFB team tried hard, but the South Americans were more determined. After a cross from Lavezzi, Higuain put in what was supposed to be a 1-0, but was correctly whistled back for an offside position. The longer the game lasted, the more both sides shied away from taking risks. The tempo of the match, which had been so racy in the first period, also dwindled with the courage. Instead, the duels became rougher: Mascherano and Aguero received warnings. And whenever a chance presented itself unexpectedly, a technical error crept in. It remained a tough struggle. Mascherano and Aguero, who had already been penalized, were lucky not to be sent off early with a yellow card when they first knocked Schweinsteiger down and then gave him a cut to the face. Schürrle put in a spirited spurt and found Götze in the center, who picked the ball off his chest and expertly fired in a left-footed volley - a dream goal at a very important time. """,
    """ Inflation is a sustained process of demonetization, which manifests itself in general price increases. One monetary unit can then constantly buy less, i.e. the purchasing power of money is permanently reduced. Inflation does not include one-time, temporary price level increases caused by unusual occurrences or price increases for specific goods or production factors. Inflation is measured by the increase in a price index that best reflects the general price level, such as the consumer price index for Germany. The percentage increase in the price index over a given period is referred to as the inflation rate. The quantity of money in the economy plays a particularly important role in the occurrence of inflation. If the quantity of goods in the economy as a whole is matched by an excessively large quantity of money, a condition for inflation is met. If the aggregate demand for goods exceeds the aggregate supply of goods, which cannot be increased in the short term, rising prices are the consequence and inflation sets in. The price increases trigger rising wages, and because of the higher income, the demand for goods increases. However, the higher wages also cause companies' costs to rise, which in turn leads to price increases for goods. """,
    """ If a traveler cancels the trip due to a virus pandemic, the tour operator is not entitled to compensation under Section 651 h (1) sentence 3 of the German Civil Code (BGB) if the traveler also cancels the trip. In addition, the lack of treatment options and vaccination justifies a right to cancel the trip free of charge. This decided the district court Stuttgart. The case was based on the following facts: A few days before the start of a multi-day flight and bus tour through Portugal in March 2020, the traveler declared the cancellation. She justified this with the spreading Corona virus. The tour operator also canceled the trip two days later. Nevertheless, it demanded payment of a cancellation fee from the traveler. Since the traveler refused to comply, a lawsuit was filed. The Stuttgart District Court ruled in favor of the traveler. The tour operator was not entitled to compensation pursuant to Section 651 h (1) sentence 3 BGB. On the one hand, the traveler could invoke extraordinary circumstances within the meaning of Section 651 h (3) of the German Civil Code. Secondly, it had to be taken into account that the tour operator had also cancelled the trip. """,
    """ To unfold the stroller, lay it flat on the ground, release the frame lock on the side and pull the handle up until the folding mechanism engages. Make sure the stroller is unfolded properly before use. To mount the rear axle, place it together with the brake system on the frame until the rear axle is audibly engaged on both sides. Hold the push button in the wheel hub and pull the wheel from the axle at the same time to remove the rear wheels and pull the wheel from the axle. To reassemble, press the push button and slide the wheel onto the axle until it locks into place. Before use, make sure that all wheels are properly mounted. The stroller is equipped with a parking brake on the rear axle. Press the brake pedal to lock the wheels. Lift the pedal to release the brake. Always block the brake when parking the stroller on uneven ground. The brake is not used to slow down when the stroller is moving and should only be locked when the stroller is at a complete standstill. The stroller is not suitable for walking. """,
    """ We encounter algorithms every day, both at work and in our leisure time, and it is impossible to imagine modern life without them. Mostly helpful but also not always harmless, algorithms are becoming increasingly important. What an algorithm is and how they shape our lives is explained in the following article. Generally speaking, an algorithm specifies a procedure for solving a problem. Based on this solution plan, input data is converted into output data in individual steps. Especially in computer science algorithms play an important role. They represent a basis of programming and are independent of a concrete programming language. Nevertheless algorithms are not to be found only in computer science or mathematics. Because algorithms are not only executed by a computer but can also be formulated and processed by humans in natural language. Our modern life is dependent on algorithms, without us always being aware of it. The areas of application of algorithms are very diverse: In the navigation system, they show us the shortest route, beat us as computer opponents in chess, control our sentence structure in Office Word or recommend a suitable partner for online dating. An algorithm can be found in many technical devices as well as behind our electronic communication. """
]

'''
- bert-base-multilingual-cased
- deepset/gbert-base
- xlm-roberta-base
- facebook/mbart-large-cc25
'''

'\n- bert-base-multilingual-cased\n- deepset/gbert-base\n- xlm-roberta-base\n- facebook/mbart-large-cc25\n'

In [17]:
# Helpers
def test_cuda():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()

    print("Device:", device)
    print("Version:", torch.__version__)


def empty_cache():
    gc.collect()
    torch.cuda.empty_cache()
    psutil.virtual_memory()

    # print(torch.cuda.get_device_properties(0).total_memory)
    # print(torch.cuda.memory_reserved(0))
    # print(torch.cuda.memory_allocated(0))


def load_tokenizer_and_model(from_checkpoint=False):
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        tokenizer_name, strip_accent=False  # add_prefix_space=True
    )

    if from_checkpoint:
        if "mbart" in model_name:
            tf2tf = transformers.AutoModelForSeq2SeqLM.from_pretrained(
                path_checkpoint
            )

        else:
            tf2tf = transformers.EncoderDecoderModel.from_pretrained(
                path_checkpoint
            )

    else:
        if "mbart" in model_name:
            tf2tf = transformers.MBartForConditionalGeneration.from_pretrained(
                model_name
            )

        else:
            tf2tf = transformers.EncoderDecoderModel.from_encoder_decoder_pretrained(
                model_name, model_name, tie_encoder_decoder=True
            )

    return tokenizer, tf2tf


def configure_model(tf2tf, tokenizer):
    tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
    tf2tf.config.bos_token_id = tokenizer.bos_token_id
    tf2tf.config.eos_token_id = tokenizer.sep_token_id
    tf2tf.config.pad_token_id = tokenizer.pad_token_id
    # tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size

    tf2tf.config.max_length = 128
    tf2tf.config.min_length = 56
    tf2tf.config.no_repeat_ngram_size = 3
    tf2tf.config.early_stopping = True
    tf2tf.config.length_penalty = 2.0
    tf2tf.config.num_beams = 2

    return tf2tf


def split_long_texts(parts, text):
    limit = 512

    if len(text) > limit:
        end_index = max([
            text.rfind(".", 0, limit),
            text.rfind("!", 0, limit),
            text.rfind("?", 0, limit)
        ])

        parts.append(text[0:end_index + 1].strip())
        text = text[end_index + 1:len(text)].strip()
        parts = split_long_texts(parts, text)

    else:
        parts.append(text)

    return parts

In [15]:
# Example
tokenizer, tf2tf = load_tokenizer_and_model(from_checkpoint=True)

if language == "german":
    corpus = corpus_german

if language == "english":
    corpus = corpus_english

if language == "multilingual":
    corpus = corpus_german + corpus_english

tf2tf = configure_model(tf2tf, tokenizer)
tf2tf.to("cuda")

test_cuda()
empty_cache()
rouge = datasets.load_metric("rouge")

cnt = 0

for text in corpus:
    cnt += 1
    parts = split_long_texts([], text)

    if len(parts) > 1:
        article = parts
        highlights = [None] * len(parts)

    else:
        parts = [text]
        article = [text] * 2
        highlights = [None] * 2

    df = pd.DataFrame({"text": article, "summary": highlights})
    test_data = datasets.arrow_dataset.Dataset.from_pandas(df)


    def generate_summary(batch):
        inputs = tokenizer(
            batch["text"],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        input_ids = inputs.input_ids.to("cuda")
        attention_mask = inputs.attention_mask.to("cuda")

        outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
        output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        batch["pred_summary"] = output_str

        return batch


    summary = test_data.map(
        generate_summary,
        batched=True,
        batch_size=batch_size
    )

    result = ""

    for i in range(0, len(parts)):
        result = result + " " + summary[i]["pred_summary"]

    with open(path_output + "summary_" + language + "_" + model_name + "_text-" + str(cnt) + ".txt", "w", encoding="utf-8") as f:
        f.write(" ".join(parts))
        f.write("\n" + "====================" + "\n")
        f.write(summary[0]["pred_summary"])
        f.write("\n" + "====================" + "\n")
        f.write(result)


Device: cuda
Version: 1.8.1+cu101
